In [17]:
'''
Aim is to find out which CN activities names has the closest match to which top 3 Singapore activity names(Since may not be exact match)
We just find for "trade" activities
Data set was created as a dummy, actual dataset was much bigger with more columns!
'''
import pandas as pd
fn = "data/daily_allocation_all.csv"
allAct_Team = pd.read_csv(fn, encoding = "unicode_escape") # the original data was encoded in ascii

print(len(allAct_Team))
allAct_Team.head()

500


,Country,OpsFunc,Team,Activities,UnitsPerHrs,Extra Col
0,CN,Trade,Trade,Active classic boxers - There's a reason why o...,7.18,0.373579
1,CN,Trade,Trade,Active sport boxer briefs - Skinning up Glory ...,0.23,0.592133
2,CN,Trade,Trade,Active sport briefs - These superbreathable no...,2.64,0.470009
3,CN,Trade,Trade,"Alpine guide pants - Skin in, climb ice, switc...",0.75,0.950653
4,CN,Trade,Trade,"Alpine wind jkt - On high ridges, steep ice an...",2.45,0.612560


In [25]:
# Filter out columns you want and then change all words to lower case if they are strings

allAct_Team_filtered = allAct_Team[['Activities','UnitsPerHrs','Country','OpsFunc','Team']]
# https://stackoverflow.com/questions/22245171/how-to-lowercase-a-pandas-dataframe-string-column-if-it-has-missing-values
allAct_Team_filtered = allAct_Team_filtered.applymap(lambda x: x.lower() if type(x)==str else x)
allAct_Team_filtered.head()

,Activities,UnitsPerHrs,Country,OpsFunc,Team
0,active classic boxers - there's a reason why o...,7.18,cn,trade,trade
1,active sport boxer briefs - skinning up glory ...,0.23,cn,trade,trade
2,active sport briefs - these superbreathable no...,2.64,cn,trade,trade
3,"alpine guide pants - skin in, climb ice, switc...",0.75,cn,trade,trade
4,"alpine wind jkt - on high ridges, steep ice an...",2.45,cn,trade,trade


In [56]:
# take out china and SG to compare later
CN_team_filtered = allAct_Team_filtered[allAct_Team_filtered['Country']=='cn']
SG_team_filtered = allAct_Team_filtered[allAct_Team_filtered['Country']=='sg']
print(CN_team_filtered)

                                            Activities  UnitsPerHrs Country  \
0    active classic boxers - there's a reason why o...         7.18      cn   
1    active sport boxer briefs - skinning up glory ...         0.23      cn   
2    active sport briefs - these superbreathable no...         2.64      cn   
3    alpine guide pants - skin in, climb ice, switc...         0.75      cn   
4    alpine wind jkt - on high ridges, steep ice an...         2.45      cn   
5    ascensionist jkt - our most technical soft she...         9.03      cn   
6    atom - a multitasker's cloud nine, the atom pl...         2.69      cn   
7    print banded betina btm - our fullest coverage...         8.64      cn   
8    baby micro d-luxe cardigan - micro d-luxe is a...         8.37      cn   
9    baby sun bucket hat - this hat goes on when th...         3.16      cn   
10   baby sunshade top - soft, stretchy polyester f...         2.76      cn   
11   baggies shorts - even baggies, our most popula.

In [57]:
# Draw out rows using this func for teams containing "trade" in "OpsFunc"
# country = ['SG','CN']
SG_trade = SG_team_filtered[SG_team_filtered['OpsFunc'].str.contains('trade')]
SG_trade = SG_trade.drop_duplicates()

CN_trade = CN_team_filtered[CN_team_filtered['OpsFunc'].str.contains('trade')]
print(CN_trade)
CN_trade = CN_trade.drop_duplicates()

                                           Activities  UnitsPerHrs Country  \
0   active classic boxers - there's a reason why o...         7.18      cn   
1   active sport boxer briefs - skinning up glory ...         0.23      cn   
2   active sport briefs - these superbreathable no...         2.64      cn   
3   alpine guide pants - skin in, climb ice, switc...         0.75      cn   
4   alpine wind jkt - on high ridges, steep ice an...         2.45      cn   
5   ascensionist jkt - our most technical soft she...         9.03      cn   
6   atom - a multitasker's cloud nine, the atom pl...         2.69      cn   
7   print banded betina btm - our fullest coverage...         8.64      cn   
8   baby micro d-luxe cardigan - micro d-luxe is a...         8.37      cn   
9   baby sun bucket hat - this hat goes on when th...         3.16      cn   
10  baby sunshade top - soft, stretchy polyester f...         2.76      cn   
11  baggies shorts - even baggies, our most popula...         5.

# This section is regular NLP protocol

In [62]:
import nltk
from nltk.corpus import stopwords
from string import punctuation
from itertools import chain
from nltk.tokenize import word_tokenize, sent_tokenize
import pickle

# its fine to get 2 dictionaries to get corpus of stopwords
customStopWords = set(stopwords.words('english') + list(punctuation))


In [90]:
# run through a few dataframes through this function built to do the regular NLP protocol
def DFName(dataframeName):
    #tokenise sentences into words
    sents1 = []
    for text in dataframeName['Activities']:
        sents1.append(sent_tokenize(text))
        
    sentsUnlist1 = [' '.join(item) for item in sents1]
    sents1 = sentsUnlist1
    
    # tokenise into words
    words1 = [word_tokenize(sent) for sent in sents1]
    dataframeName = dataframeName.assign(tokenised = words1)
    # remove stopwords 
    wordsWOStopwords = []
    for text in dataframeName['tokenised']:
        # remove "/" from words
        splitted =[]
        for word in text:
            w = word.split("/")
            splitted.append(w)
        
        # unlist "splitted" from list of lists
        text = list(chain(*splitted))
        text = " ".join(text)
        wordsWOStopwords.append([word for word in word_tokenize(text.lower()) if word not in customStopWords])
        
    dataframeName = dataframeName.assign(removeStopwords= wordsWOStopwords)
    
    # unlist the list
    wordsWOStopwords = list(chain(*wordsWOStopwords)) # unlist a list of lists
    
    # no need to lemmetize words as this column is all names only
    # merge each row of "sentence" back to 1 sentence instead of list
    lenDF = len(dataframeName)
    listMergeAll = []
    for i in range(0,lenDF):
        stopWorded = dataframeName.removeStopwords.iloc[i]
        mergestr = ' '.join(stopWorded)
        listMergeAll.append(mergestr)
    dataframeName = dataframeName.assign(mergeAll = listMergeAll)
    return(dataframeName)
    print(dataframeName)

In [92]:
# get the pre-processed dataframe for CN and SG
CN_old_activities = DFName(CN_trade)
SG_old_activities = DFName(SG_trade)
SG_old_activities

,Activities,UnitsPerHrs,Country,OpsFunc,Team,tokenised,removeStopwords,mergeAll
369,stand up shorts-7 in. - they've been around fo...,3.54,sg,trade,trade security,"[stand, up, shorts-7, in, ., -, they, 've, bee...","[stand, shorts-7, 've, around, 30-plus, years,...",stand shorts-7 've around 30-plus years re-env...
370,sticks 'n stones morocco poster - patagonia ro...,2.40,sg,trade,trade security,"[sticks, ', n, stones, morocco, poster, -, pat...","[sticks, n, stones, morocco, poster, patagonia...",sticks n stones morocco poster patagonia rock ...
371,"peregrine t-shirt - a born wanderer, the once-...",0.81,sg,trade,trade security,"[peregrine, t-shirt, -, a, born, wanderer, ,, ...","[peregrine, t-shirt, born, wanderer, once-enda...",peregrine t-shirt born wanderer once-endangere...
372,permit silhouette t-shirt - the permit's scyth...,4.34,sg,trade,trade security,"[permit, silhouette, t-shirt, -, the, permit, ...","[permit, silhouette, t-shirt, permit, 's, scyt...",permit silhouette t-shirt permit 's scythe-sha...
373,pique fleece 1/4 zip - made from plush texture...,8.63,sg,trade,trade security,"[pique, fleece, 1/4, zip, -, made, from, plush...","[pique, fleece, 1, 4, zip, made, plush, textur...",pique fleece 1 4 zip made plush textured fleec...
374,pique vitaliti polo shirt - the perfect summer...,7.42,sg,trade,trade security,"[pique, vitaliti, polo, shirt, -, the, perfect...","[pique, vitaliti, polo, shirt, perfect, summer...",pique vitaliti polo shirt perfect summer fabri...
375,polo shirt - pulling on a polo opens the door ...,7.24,sg,trade,trade security,"[polo, shirt, -, pulling, on, a, polo, opens, ...","[polo, shirt, pulling, polo, opens, door, poss...",polo shirt pulling polo opens door possibiliti...
376,"quiet please t-shirt - thoreau said that ""sile...",0.80,sg,trade,trade security,"[quiet, please, t-shirt, -, thoreau, said, tha...","[quiet, please, t-shirt, thoreau, said, ``, si...",quiet please t-shirt thoreau said `` silence u...
377,"r1 full-zip jkt - it stretches, breathes, wick...",4.94,sg,trade,trade security,"[r1, full-zip, jkt, -, it, stretches, ,, breat...","[r1, full-zip, jkt, stretches, breathes, wicks...",r1 full-zip jkt stretches breathes wicks warms...
378,lw guide pants - a backcountry version of our ...,3.68,sg,trade,trade security,"[lw, guide, pants, -, a, backcountry, version,...","[lw, guide, pants, backcountry, version, immen...",lw guide pants backcountry version immensely p...


# Jaccard Similarity Score

In [72]:
import numpy as np
def jaccard_similarity(list1,list2):
    intersection = len(list(set(list1).intersection(list2)))
    try:
        union = (len(set(list1)) + len(set(list2))) - intersection
        return float(intersection/union)
    except ZeroDivisionError:
        return(np.nan)
    

In [51]:
# # input my 2 lists into Jaccard_similarity
# jaccard_similarity(CN_old_activities['mergeAll'].tolist(),
#                   SG_old_activities['mergeAll'].tolist())

In [77]:
# apply Jaccard similarity sentence by sentence across all sentence in CN and SG dataframes
import numpy
a = SG_old_activities.removeStopwords.tolist()
a_name = SG_old_activities.mergeAll.tolist()

b= CN_old_activities.removeStopwords.tolist()
b_name = CN_old_activities.mergeAll.tolist()

# create matrix then feed in elements later
c= numpy.zeros(shape=(len(a), len(b)))
df_c = pd.DataFrame(c, columns = b_name)

# add a column "new1" to include SG activity names 
df_c['new1'] = a_name
df_c.set_index('new1')

for i in range(0, len(a_name)):
    for j in range(0, len(b_name)):
        df_c.iloc[i,j] = jaccard_similarity(a[i],b[j])
        
df_c

,active classic boxers 's reason boxers cult favorite keep cool especially sticky situations quick-drying lightweight underwear takes minimal space travel pack exposed brushed waistband offers next-to-skin softness five-panel construction traditional boxer back classic fit functional fly made 3.7-oz 100 recycled polyester moisture-wicking performance inseam size 4 1 2 `` recyclable common threads recycling program br br b details b ul li `` silky capilene 1 fabric ultralight breathable quick-to-dry `` li li `` exposed brushed elastic waistband comfort `` li li 5-panel construction traditional boxer back li li `` inseam size 4 1 2 `` `` `` li ul br br b fabric b 3.7-oz 100 all-recycled polyester gladiodor natural odor control garment recyclable common threads recycling program br br b weight b 99 g 3.5 oz br br made mexico,active sport boxer briefs skinning glory requires enough movement without boxers deciding poach route form-fitting active sport boxer briefs made breathable 93 polyester 71 recycled fabric 's fast-wicking dries quickly 7 spandex stretch seamless waistband soft leg edges wo n't roll bind gusseted flat-sewn 6 `` inseam size offset prevent inner-thigh chafe fly-free smooth front panel recyclable common threads recycling program br br b details b ul li `` stretch mesh provides support open-weave mesh airflow wicks efficiently dries fast `` li li seamless construction li li `` flat-sewn gusseted inseam set forward prevent inner-thigh chafe `` li li fly-free support li li `` inseam size 6 `` `` `` li ul br br b fabric b `` 4.6-oz 93 polyester 71 recycled 7 spandex moisture-wicking performance recyclable common threads recycling program `` br br b weight b 60 g 2.1 oz br br made israel,active sport briefs superbreathable no-fly briefs minimalist 's choice high-octane endeavors made blend fast-wicking quick-drying 93 polyester 71 recycled 7 spandex stretch-mesh support open mesh cooling airflow soft edging leg openings seamless waist wo n't roll create friction layers smooth front panel opacity recyclable common threads recycling program br br b details b ul li `` stretch mesh provides support open-weave mesh airflow wicks efficiently dries fast `` li li seamless construction li li leg openings half-inch self-binding comfort li li fly-free support li ul br br b fabric b `` 4.6-oz 93 polyester 71 recycled 7 spandex moisture-wicking performance recyclable common threads recycling program `` br br b weight b 49 g 1.7 oz br br made israel,alpine guide pants skin climb ice switch rock traverse knife-edge ridge boogie back durable weather-resistant breathable soft-shell pants keep stride every mountain endeavor midweight stretch-woven polyester wo n't restrict moves brushed interior maintains next-to-skin comfort 2-way zippered fly keeps things easy harness gusseted zippered cuffs 2-position snaps tie-down loops water-resistant zippered pockets two front two thigh one back hip recyclable common threads recycling program br br b details b ul li durable stretch-woven polyester 47 recycled dwr durable water repellent finish water- wind-resistant highly breathable li li brushed interior next-to-skin comfort li li waistband belt loops elastic back li li 2-way zippered fly li li `` external pockets two front slash two glued-on thigh one back hip highly water-resistant dwr-finished zippers `` li li `` gusseted zippered cuff 2-position adjustable settings tie-down loops `` li ul br br b fabric b `` 7.6-oz 90-denier 92 polyester 47 recycled 8 spandex deluge dwr finish recyclable common threads recycling program `` br br b weight b 597 g 20.7 oz br br made vietnam,alpine wind jkt high ridges steep ice anything alpine jacket serves true `` best worlds `` staple excels stand-alone shell blustery rock climbs cool-weather trail runs high-output ski tours conditions ice alpine climbing functions lightly insulated windshirt approach well frictionless midlayer 's time bundle tie-in polyester ripstop shell deluge dwr durable water repellent

In [81]:
# sort according to each SG activity top 3 per CN activity, then take its index
counter = 0
dfNew = pd.DataFrame(columns = ['CN_task','SG_task'])
q = pd.DataFrame(columns = [])
# remember that last column is made by us "new1", remove in the loop
for column in df_c.columns[:-1]:
    dfObj = df_c.sort_values(by = column, ascending = False)
    print(dfObj)
    colname = df_c.columns[counter]
    
    # take top 3
    b = dfObj['new1'][0:3]

    dfNew = pd.DataFrame(b)
    dfNew['CN'] = [colname]*3
    
    counter = counter + 1 
    q = q.append(dfNew)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [85]:
# put in correct order for dataframe, where CN comes first then SG
new_order = [1,0]

# rename column header from new1 to SG
q.rename(columns={'new1':'SG'}, inplace = True)

In [96]:
# to append back the 'UnitsPerHrs' we append back the df "SG_old_activities" and "CN_old_activities"
merged = pd.merge(left = q , right = CN_old_activities , how = 'left',  left_on = 'CN', right_on = 'mergeAll')
merged

,SG,CN,Activities,UnitsPerHrs,Country,OpsFunc,Team,tokenised,removeStopwords,mergeAll
0,merino 2 bottoms stop-and-go pace like alpine ...,active classic boxers 's reason boxers cult fa...,active classic boxers - there's a reason why o...,7.18,cn,trade,trade,"[active, classic, boxers, -, there, 's, a, rea...","[active, classic, boxers, 's, reason, boxers, ...",active classic boxers 's reason boxers cult fa...
1,relax fit organic ctn jeans-reg everyday go-to...,active classic boxers 's reason boxers cult fa...,active classic boxers - there's a reason why o...,7.18,cn,trade,trade,"[active, classic, boxers, -, there, 's, a, rea...","[active, classic, boxers, 's, reason, boxers, ...",active classic boxers 's reason boxers cult fa...
2,relax fit organic ctn jeans-long everyday go-t...,active classic boxers 's reason boxers cult fa...,active classic boxers - there's a reason why o...,7.18,cn,trade,trade,"[active, classic, boxers, -, there, 's, a, rea...","[active, classic, boxers, 's, reason, boxers, ...",active classic boxers 's reason boxers cult fa...
3,r1 full-zip jkt stretches breathes wicks warms...,active sport boxer briefs skinning glory requi...,active sport boxer briefs - skinning up glory ...,0.23,cn,trade,trade,"[active, sport, boxer, briefs, -, skinning, up...","[active, sport, boxer, briefs, skinning, glory...",active sport boxer briefs skinning glory requi...
4,merino 2 bottoms stop-and-go pace like alpine ...,active sport boxer briefs skinning glory requi...,active sport boxer briefs - skinning up glory ...,0.23,cn,trade,trade,"[active, sport, boxer, briefs, -, skinning, up...","[active, sport, boxer, briefs, skinning, glory...",active sport boxer briefs skinning glory requi...
5,merino 2 crew soft long-sleeved tee work sweat...,active sport boxer briefs skinning glory requi...,active sport boxer briefs - skinning up glory ...,0.23,cn,trade,trade,"[active, sport, boxer, briefs, -, skinning, up...","[active, sport, boxer, briefs, skinning, glory...",active sport boxer briefs skinning glory requi...
6,r1 full-zip jkt stretches breathes wicks warms...,active sport briefs superbreathable no-fly bri...,active sport briefs - these superbreathable no...,2.64,cn,trade,trade,"[active, sport, briefs, -, these, superbreatha...","[active, sport, briefs, superbreathable, no-fl...",active sport briefs superbreathable no-fly bri...
7,lw everyday socks essential part daily grind e...,active sport briefs superbreathable no-fly bri...,active sport briefs - these superbreathable no...,2.64,cn,trade,trade,"[active, sport, briefs, -, these, superbreatha...","[active, sport, briefs, superbreathable, no-fl...",active sport briefs superbreathable no-fly bri...
8,pique vitaliti polo shirt perfect summer fabri...,active sport briefs superbreathable no-fly bri...,active sport briefs - these superbreathable no...,2.64,cn,trade,trade,"[active, sport, briefs, -, these, superbreatha...","[active, sport, briefs, superbreathable, no-fl...",active sport briefs superbreathable no-fly bri...
9,lw guide pants backcountry version immensely p...,alpine guide pants skin climb ice switch rock ...,"alpine guide pants - skin in, climb ice, switc...",0.75,cn,trade,trade,"[alpine, guide, pants, -, skin, in, ,, climb, ...","[alpine, guide, pants, skin, climb, ice, switc...",alpine guide pants skin climb ice switch rock ...


In [97]:
# to append back the 'UnitsPerHrs' we append back the df "SG_old_activities" and "CN_old_activities"
merged_left1 = pd.merge(left = merged , right = SG_old_activities , how = 'left',  left_on = 'SG', right_on = 'mergeAll')
merged_left1

,SG,CN,Activities_x,UnitsPerHrs_x,Country_x,OpsFunc_x,Team_x,tokenised_x,removeStopwords_x,mergeAll_x,Activities_y,UnitsPerHrs_y,Country_y,OpsFunc_y,Team_y,tokenised_y,removeStopwords_y,mergeAll_y
0,merino 2 bottoms stop-and-go pace like alpine ...,active classic boxers 's reason boxers cult fa...,active classic boxers - there's a reason why o...,7.18,cn,trade,trade,"[active, classic, boxers, -, there, 's, a, rea...","[active, classic, boxers, 's, reason, boxers, ...",active classic boxers 's reason boxers cult fa...,"merino 2 bottoms - a stop-and-go pace, like th...",7.19,sg,trade,trade ops,"[merino, 2, bottoms, -, a, stop-and-go, pace, ...","[merino, 2, bottoms, stop-and-go, pace, like, ...",merino 2 bottoms stop-and-go pace like alpine ...
1,relax fit organic ctn jeans-reg everyday go-to...,active classic boxers 's reason boxers cult fa...,active classic boxers - there's a reason why o...,7.18,cn,trade,trade,"[active, classic, boxers, -, there, 's, a, rea...","[active, classic, boxers, 's, reason, boxers, ...",active classic boxers 's reason boxers cult fa...,relax fit organic ctn jeans-reg - the everyday...,6.03,sg,trade,trade ops,"[relax, fit, organic, ctn, jeans-reg, -, the, ...","[relax, fit, organic, ctn, jeans-reg, everyday...",relax fit organic ctn jeans-reg everyday go-to...
2,relax fit organic ctn jeans-long everyday go-t...,active classic boxers 's reason boxers cult fa...,active classic boxers - there's a reason why o...,7.18,cn,trade,trade,"[active, classic, boxers, -, there, 's, a, rea...","[active, classic, boxers, 's, reason, boxers, ...",active classic boxers 's reason boxers cult fa...,relax fit organic ctn jeans-long - the everyda...,4.23,sg,trade,trade ops,"[relax, fit, organic, ctn, jeans-long, -, the,...","[relax, fit, organic, ctn, jeans-long, everyda...",relax fit organic ctn jeans-long everyday go-t...
3,r1 full-zip jkt stretches breathes wicks warms...,active sport boxer briefs skinning glory requi...,active sport boxer briefs - skinning up glory ...,0.23,cn,trade,trade,"[active, sport, boxer, briefs, -, skinning, up...","[active, sport, boxer, briefs, skinning, glory...",active sport boxer briefs skinning glory requi...,"r1 full-zip jkt - it stretches, breathes, wick...",4.94,sg,trade,trade security,"[r1, full-zip, jkt, -, it, stretches, ,, breat...","[r1, full-zip, jkt, stretches, breathes, wicks...",r1 full-zip jkt stretches breathes wicks warms...
4,merino 2 bottoms stop-and-go pace like alpine ...,active sport boxer briefs skinning glory requi...,active sport boxer briefs - skinning up glory ...,0.23,cn,trade,trade,"[active, sport, boxer, briefs, -, skinning, up...","[active, sport, boxer, briefs, skinning, glory...",active sport boxer briefs skinning glory requi...,"merino 2 bottoms - a stop-and-go pace, like th...",7.19,sg,trade,trade ops,"[merino, 2, bottoms, -, a, stop-and-go, pace, ...","[merino, 2, bottoms, stop-and-go, pace, like, ...",merino 2 bottoms stop-and-go pace like alpine ...
5,merino 2 crew soft long-sleeved tee work sweat...,active sport boxer briefs skinning glory requi...,active sport boxer briefs - skinning up glory ...,0.23,cn,trade,trade,"[active, sport, boxer, briefs, -, skinning, up...","[active, sport, boxer, briefs, skinning, glory...",active sport boxer briefs skinning glory requi...,"merino 2 crew - in this soft long-sleeved tee,...",5.01,sg,trade,trade ops,"[merino, 2, crew, -, in, this, soft, long-slee...","[merino, 2, crew, soft, long-sleeved, tee, wor...",merino 2 crew soft long-sleeved tee work sweat...
6,r1 full-zip jkt stretches breathes wicks warms...,active sport briefs superbreathable no-fly bri...,active sport briefs - these superbreathable no...,2.64,cn,trade,trade,"[active, sport, briefs, -, these, superbreatha...","[active, sport, briefs, superbreathable, no-fl...",active sport briefs superbreathable no-fly bri...,"r1 full-zip jkt - it stretches, breathes, wick...",4.94,sg,trade,trade security,"[r1, full-zip, jkt, -, it, stretches, ,, breat...","[r1, f

In [104]:
# take only columns i want
columnsWanted = merged_left1[['CN','UnitsPerHrs_x','Team_x','SG','UnitsPerHrs_y','Team_y']]
columnsWanted = columnsWanted.rename(index=str, columns = {'UnitsPerHrs_x': 'UnitsPerHrs_CN','Team_x':'Team_CN','UnitsPerHrs_y':'UnitsPerHrs_SG','Team_y':'Team_SG'})
columnsWanted.head()

,CN,UnitsPerHrs_CN,Team_CN,SG,UnitsPerHrs_SG,Team_SG
0,active classic boxers 's reason boxers cult fa...,7.18,trade,merino 2 bottoms stop-and-go pace like alpine ...,7.19,trade ops
1,active classic boxers 's reason boxers cult fa...,7.18,trade,relax fit organic ctn jeans-reg everyday go-to...,6.03,trade ops
2,active classic boxers 's reason boxers cult fa...,7.18,trade,relax fit organic ctn jeans-long everyday go-t...,4.23,trade ops
3,active sport boxer briefs skinning glory requi...,0.23,trade,r1 full-zip jkt stretches breathes wicks warms...,4.94,trade security
4,active sport boxer briefs skinning glory requi...,0.23,trade,merino 2 bottoms stop-and-go pace like alpine ...,7.19,trade ops


In [113]:
# Now find out the difference in UnitPerHrs by SG vs CN
import math
difference_SG_CN = []
for row in columnsWanted.iterrows():
    try:
        diff = (row[1]['UnitsPerHrs_SG']- row[1]['UnitsPerHrs_CN'])/row[1]['UnitsPerHrs_SG']
        diff = diff *100
    except ZeroDivisionError:
        diff = 0
        diff = diff*100
    difference_SG_CN.append(diff)

print(len(columnsWanted))
columnsWanted =  columnsWanted.assign(difference_SG_CN= difference_SG_CN)
columnsWanted.to_csv('CN_SG_DiffUnitsPerHrs.csv')

39


In [114]:
columnsWanted

,CN,UnitsPerHrs_CN,Team_CN,SG,UnitsPerHrs_SG,Team_SG,difference_SG_CN
0,active classic boxers 's reason boxers cult fa...,7.18,trade,merino 2 bottoms stop-and-go pace like alpine ...,7.19,trade ops,0.139082
1,active classic boxers 's reason boxers cult fa...,7.18,trade,relax fit organic ctn jeans-reg everyday go-to...,6.03,trade ops,-19.071310
2,active classic boxers 's reason boxers cult fa...,7.18,trade,relax fit organic ctn jeans-long everyday go-t...,4.23,trade ops,-69.739953
3,active sport boxer briefs skinning glory requi...,0.23,trade,r1 full-zip jkt stretches breathes wicks warms...,4.94,trade security,95.344130
4,active sport boxer briefs skinning glory requi...,0.23,trade,merino 2 bottoms stop-and-go pace like alpine ...,7.19,trade ops,96.801113
5,active sport boxer briefs skinning glory requi...,0.23,trade,merino 2 crew soft long-sleeved tee work sweat...,5.01,trade ops,95.409182
6,active sport briefs superbreathable no-fly bri...,2.64,trade,r1 full-zip jkt stretches breathes wicks warms...,4.94,trade security,46.558704
7,active sport briefs superbreathable no-fly bri...,2.64,trade,lw everyday socks essential part daily grind e...,6.04,trade security,56.291391
8,active sport briefs superbreathable no-fly bri...,2.64,trade,pique vitaliti polo shirt perfect summer fabri...,7.42,trade security,64.420485
9,alpine guide pants skin climb ice switch rock ...,0.75,trade,lw guide pants backcountry version immensely p...,3.68,trade security,79.619565
